In [1]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    AIMessage,
    ToolMessage,
    FunctionMessage,
)
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here

In [2]:
import re
import time
import pandas as pd
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib

module = importlib.import_module("gsheet_handler")
importlib.reload(module)
ggl = module.GSheetHandler("google_sheet_key.json")

# data_raw = pd.read_csv('/home/sftpuser1/uploads/data.csv')
# data_raw

In [3]:
job_vacancy = ggl.extract_data_from_google_sheet(
    "https://docs.google.com/spreadsheets/d/18zb9tDE4DiGrMbRt4FMbW7GW6VRzxYPwMr5S9BGJS-I/edit?gid=0#gid=0",
    0,
)
job_vacancy_text = job_vacancy.to_string()
job_vacancy_text

'Empty DataFrame\nColumns: [Cистемный администратор \n\nТребуемый опыт работы: от 2 лет\nПолная занятость, полный день\n\nОбязанности:\nТехническая поддержка и сопровождение пользователей, АРМ сотрудников;\nЗнание на уровне администратора семейства ОС MS Windows 10;\nПоддержка работы основных корпоративных сервисов на рабочих станциях (электронная почта, печать, файловый сервис, банк-клиенты, бизнес-приложения, удаленный доступ (RDP) и т.д.); \nРабота с ЭЦП \nБазовое администрирование ИТ-сервисов на оборудовании под управлением Windows Server, СУБД MS SQL Server, Linux.\nСопровождение виртуальной среды на базе VMware; \nАдминистрирование корпоративной антивирусной системы (Kaspersky), \nВыявление и устранение неисправностей в поддерживаемой инфраструктуре;  \nУстановка, настройка периферийного и сетевого оборудования (принтеры, копиры, МФУ); \nНастройка телефонов (SIP), администрирование Asterisk;\nАдминистрирование корпоративной сети (Cisco);\nПоддержка актуальности версий серверных О

In [4]:
data_raw = ggl.extract_data_from_google_sheet(
    "https://docs.google.com/spreadsheets/d/18zb9tDE4DiGrMbRt4FMbW7GW6VRzxYPwMr5S9BGJS-I/edit?gid=0#gid=0",
    1,
)
data_raw

,File Name,Text
0,Рязанцев Евгений.docx,\nОпыт работы\nКлючевые навыки\n• Ответственно...
1,Славин Роман Михайлович.pdf,"Славин Роман Михайлович\nМужчина, 29 лет, роди..."
2,Пастухов Игорь.docx,\nОпыт работы\nКлючевые навыки\n• Active Direc...
3,Богачев Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Администриро...
4,Карандашов Алексей.docx,\nОпыт работы\n\nДекабрь.2022 —н.в.\tРозничная...
5,Макряшин Геогрий.docx,\nОпыт работы\nКлючевые навыки\n• Исполнительн...
6,Карлов Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Знания уровн...
7,Антонов Иван.docx,\nОпыт работы\nКлючевые навыки\n• MS Windows 2...
8,Сотниченко Андрей.docx,\nОпыт работы\nКлючевые навыки\n• Linux ...
9,Бунеско Андрей Александрович.pdf,"Бунеско Андрей Александрович\nМужчина, 35 лет,..."


In [5]:
# df = data_raw.copy()
# df['article_time'] = pd.to_datetime(df['article_time'])
# filtered_df = df[df['article_time'] == '2024-05-29']
filtered_df = data_raw.copy()

In [6]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open("config.json") as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"

In [7]:
from datetime import datetime
from langchain import hub


def evaluate_news_article(job_vacancy_text, resume):
    llm = ChatOpenAI(
        temperature=0,
        # model="gpt-3.5-turbo",
        model="gpt-4o",
    )
    prompt = hub.pull("mlenparrot/nlmk_hr_grade")
    today = datetime.now().strftime("%Y-%m-%d")
    params = {"job_vacancy_text": job_vacancy_text, "resume": resume, "today": today}
    chain = prompt | llm
    result = chain.invoke(params)
    return result


result = evaluate_news_article(
    job_vacancy_text="ищем старушку попрыгушку", resume="мальчик одуванчик"
)

print(result)

{'Experience_match': 0, 'Employment_tenure': 0, 'Relevant_education': 0}


In [8]:
def apply_evaluation(row):
    result = evaluate_news_article(
        job_vacancy_text=job_vacancy_text,
        resume=row["Text"],
    )
    return pd.Series(result)

In [9]:
from tqdm import tqdm

tqdm.pandas()
data_news = filtered_df.copy()
evaluation_results = data_news.progress_apply(apply_evaluation, axis=1)

100%|██████████| 19/19 [00:24<00:00,  1.28s/it]


In [10]:
data_news_rated = pd.concat(
    [data_news, evaluation_results], axis=1
)  # .drop(['reference_time', 'author', 'tags'], axis=1)
data_news_rated

,File Name,Text,Experience_match,Employment_tenure,Relevant_education
0,Рязанцев Евгений.docx,\nОпыт работы\nКлючевые навыки\n• Ответственно...,0,0,0
1,Славин Роман Михайлович.pdf,"Славин Роман Михайлович\nМужчина, 29 лет, роди...",3,4,0
2,Пастухов Игорь.docx,\nОпыт работы\nКлючевые навыки\n• Active Direc...,0,0,0
3,Богачев Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Администриро...,0,0,0
4,Карандашов Алексей.docx,\nОпыт работы\n\nДекабрь.2022 —н.в.\tРозничная...,4,1,0
5,Макряшин Геогрий.docx,\nОпыт работы\nКлючевые навыки\n• Исполнительн...,0,0,4
6,Карлов Сергей.docx,\nОпыт работы\nКлючевые навыки\n• Знания уровн...,0,0,0
7,Антонов Иван.docx,\nОпыт работы\nКлючевые навыки\n• MS Windows 2...,0,0,0
8,Сотниченко Андрей.docx,\nОпыт работы\nКлючевые навыки\n• Linux ...,0,0,0
9,Бунеско Андрей Александрович.pdf,"Бунеско Андрей Александрович\nМужчина, 35 лет,...",5,4,5


In [11]:
integer_columns = data_news_rated.select_dtypes(include=["int64"]).columns
sum_values = data_news_rated[integer_columns].sum(axis=1)

data_news_rated["rate"] = sum_values
data_news_rated = data_news_rated.sort_values("rate", ascending=False).reset_index(
    drop=True
)
data_news_rated

,File Name,Text,Experience_match,Employment_tenure,Relevant_education,rate
0,Бунеско Андрей Александрович.pdf,"Бунеско Андрей Александрович\nМужчина, 35 лет,...",5,4,5,14
1,Клыковский Вячеслав Витальевич.pdf,Клыковский Вячеслав Витальевич\nМужчина\n+7 (9...,3,5,2,10
2,Славин Роман Михайлович.pdf,"Славин Роман Михайлович\nМужчина, 29 лет, роди...",3,4,0,7
3,Карандашов Алексей.docx,\nОпыт работы\n\nДекабрь.2022 —н.в.\tРозничная...,4,1,0,5
4,Макряшин Геогрий.docx,\nОпыт работы\nКлючевые навыки\n• Исполнительн...,0,0,4,4
5,Ус Владимир.docx,\nОпыт работы\nКлючевые навыки\n• Windows Serv...,3,0,0,3
6,Биленко Руслан.docx,\nОпыт работы\nКлючевые навыки\n• • Установка ...,2,0,0,2
7,Полушкин Виталий.docx,\nОпыт работы\nКлючевые навыки\n• Active Direc...,0,0,0,0
8,Полостюк Николай.docx,\nОпыт работы\nКлючевые навыки\n• VMware ...,0,0,0,0
9,Буханцев Александр.docx,\nОпыт работы\nКлючевые навыки\n• MS Excel ...,0,0,0,0


In [12]:
ggl.update_gsheet_with_df(
    data_news_rated,
    "https://docs.google.com/spreadsheets/d/18zb9tDE4DiGrMbRt4FMbW7GW6VRzxYPwMr5S9BGJS-I/edit?gid=491589845#gid=491589845",
    2,
)

In [13]:
data_news_rated[:3].keys()

Index(['File Name', 'Text', 'Experience_match', 'Employment_tenure',
       'Relevant_education', 'rate'],
      dtype='object')

In [14]:
0()

<>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_226945/1350904145.py:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  0()
/tmp/ipykernel_226945/1350904145.py:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  0()
/tmp/ipykernel_226945/1350904145.py:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  0()


TypeError: 'int' object is not callable

In [ ]:
data_news_summary = (
    data_news_rated  # [:3][['link', 'news_date', 'news_topic', 'news_text']]
)
data_news_summary

In [ ]:
hub.pull("mlenparrot/nlmk_grade")

In [ ]:
def summarize_news_article(counts, news_title, news_text):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_summary")
    params = {
        "news_title": news_title,
        "counts": counts,
        "news_text": news_text,
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

In [ ]:
pd.set_option("display.max_colwidth", None)
news_text = data_news_summary.loc[0, "news_text"]  # .to_string()
news_title = data_news_summary.loc[0, "news_topic"]
pd.reset_option("display.max_colwidth")
news_text

In [ ]:
summarize_news_article(500, news_title, news_text)

In [ ]:
# Инициализация новых столбцов
data_news_summary["summary"] = ""
data_news_summary["summary_title"] = ""

# Итерация по строкам DataFrame с использованием tqdm
for index, row in tqdm(data_news_summary.iterrows(), total=data_news_summary.shape[0]):
    result = summarize_news_article(500, row["news_topic"], row["news_text"])
    data_news_summary.at[index, "summary"] = result["news_summary"]
    data_news_summary.at[index, "summary_title"] = result["news_topic"]

# Переупорядочивание столбцов для удобства
# data_news_summary = data_news_summary[['link', 'news_date', 'news_topic', 'news_title', 'news_text', 'summary', 'summary_title']]

In [ ]:
data_news_summary

In [ ]:
ggl.update_gsheet_with_df(
    data_news_summary,
    "https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing",
    1,
)

In [ ]:
data_news_summary.loc[0, "summary"]